In [19]:
#conda activate AP1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
from scipy.stats import skew, kurtosis

import pyfolio as pf
import empyrical as emp

In [20]:
df = pd.read_excel('data_nn.xlsx')
#df.to_pickle("data_nn.xlsx")

In [ ]:
# Set the first column as the date index
df.set_index(df.columns[0], inplace=True)

# Convert the index to string and then to DatetimeIndex format
df.index = pd.to_datetime(df.index.astype(str))

# Filter the data for the last ten years
df_last_10_years = df.loc[df.index > "2020-01-02"]

# Apply rolling sum with a window of 252 and require at least 126 non-NaN values
df_rolling_sum = df_last_10_years.rolling(window=252, min_periods=int(252//2)).sum()

# Forward-fill NaN values, but limit this to a maximum of 5 consecutive fills
df_filled = df_last_10_years.ffill(limit=5)

# Drop any remaining NaN values that still exist after the forward-fill operation
df_cleaned = df_filled.dropna()

#return back original name to not interruppt code.
df_last_10_years = df_cleaned




In [75]:
def refactored_advanced_features(df_returns):
    """
    Refactored computation of advanced financial features to reduce DataFrame fragmentation.
    """
    skew = {}
    kurtosis = {}
    max_drawdown = {}
    volatility = {}
    vaR = {}
    momentum = {}
    avg_return = {}
    rsi = {}

        
        # 1. Skewness
    print("Skewness")
    for window in [20, 40, 60, 100, 180, 240, 360, 480]:
        skew[window] = df_returns.rolling(window).skew()

        # 2. Kurtosis
    print("Kurtosis")
    for window in [20, 40, 60, 100, 180, 240, 360, 480]:
        kurtosis[window]=df_returns.rolling(window).kurt()
    
    # 3. Maximum drawdown
    print("Maximum drawdown")
    for window in [20, 40, 60, 100, 180, 240, 360, 480]:
        max_drawdown[window] = df_returns.rolling(window).apply(emp.max_drawdown, raw=True)
    
    # 4. Volatility
    print("Volatility")
    for window in [20, 40, 60, 100, 180, 240, 360, 480]:
        volatility[window] = df_returns.rolling(window).std()*(252**0.5)
    
    # 5. Value at Risk
    print("Value at Risk")
    for window in [20, 40, 60, 100, 180, 240, 360, 480]:
        vaR[window] = df_returns.rolling(window).apply(emp.value_at_risk, raw=True)
    
    # 6. Momentum
    print("Momentum")
    for window in [20, 40, 60, 100, 180, 240, 360, 480]:
        momentum[window] = df_returns.rolling(window).sum() # ?

    print("Average Return")
    for window in [20, 40, 60, 100, 180, 240, 360, 480]:
        avg_return[window] = df_returns.rolling(window).mean()
    
    return skew, kurtosis, max_drawdown, volatility, vaR, momentum, avg_return

# This function reduces DataFrame fragmentation by constructing all columns and concatenating them at once.

# Läs tommys mex hur de gjorde reversal, sen implementera det. Fixa windows size till vad de hade i rapporten.
# skew[20].head() 

In [ ]:
# Call the function and capture the output
skew, kurtosis, max_drawdown, volatility, vaR, momentum, avg_return = refactored_advanced_features(df_last_10_years)


In [107]:
# Reset the feature DataFrames list
features_df_list = []

# Create individual lists for each feature's DataFrame
skew_df_list = [] 
kurtosis_df_list = []
max_drawdown_df_list = []
volatility_df_list = []
vaR_df_list = []
momentum_df_list = []
avg_return_df_list = []

# Windows configuration
windows = [20, 40, 60, 100, 180, 240, 360, 480]

# Iterate through each feature dictionary and create a DataFrame
for feature_name, feature_dict in [('skew', skew), ('kurtosis', kurtosis), ('max_drawdown', max_drawdown), 
                                   ('volatility', volatility), ('vaR', vaR), ('momentum', momentum), ('avg_return', avg_return)]:
    # Only keep the windows that are present for each feature
    relevant_windows = windows if feature_name != 'kurtosis' else windows[:-1]
    feature_df = pd.concat({f'{feature_name}_{window}': feature_dict[window] for window in relevant_windows}, axis=1)
    
    # Append the individual DataFrame to the corresponding feature list
    if feature_name == 'skew':
        skew_df_list.append(feature_df)
    elif feature_name == 'kurtosis':
        kurtosis_df_list.append(feature_df)
    elif feature_name == 'max_drawdown':
        max_drawdown_df_list.append(feature_df)
    elif feature_name == 'volatility':
        volatility_df_list.append(feature_df)
    elif feature_name == 'vaR':
        vaR_df_list.append(feature_df)
    elif feature_name == 'momentum':
        momentum_df_list.append(feature_df)
    elif feature_name == 'avg_return':
        avg_return_df_list.append(feature_df)
    
    # Add the DataFrame to the main list
    features_df_list.append(feature_df)


# Concatenate all feature DataFrames into a single DataFrame
features_df = pd.concat(features_df_list, axis=1)

# Concatenate all feature DataFrames into a single DataFrame for each feature
if len(skew_df_list) > 1:
    skew_df = pd.concat(skew_df_list, axis=1)
if len(kurtosis_df_list) > 1:
    kurtosis_df = pd.concat(kurtosis_df_list, axis=1)
if len(max_drawdown_df_list) > 1:
    max_drawdown_df = pd.concat(max_drawdown_df_list, axis=1)
if len(volatility_df_list) > 1:
    volatility_df = pd.concat(volatility_df_list, axis=1)
if len(vaR_df_list) > 1:
    vaR_df = pd.concat(vaR_df_list, axis=1)
if len(momentum_df_list) > 1:
    momentum_df = pd.concat(momentum_df_list, axis=1)
if len(avg_return_df_list) > 1:
    avg_return_df = pd.concat(avg_return_df_list, axis=1)



# The individual lists for each feature now contain their respective DataFrames
# And features_df_list contains all the feature DataFrames
# Let's print the first item of each sublist to confirm
#print("Skew DataFrame:\n", skew_df_list[0].tail(), "\n")
#print("Kurtosis DataFrame:\n", kurtosis_df_list[0].tail(), "\n")
#print("Max Drawdown DataFrame:\n", max_drawdown_df_list[0].tail(), "\n")
#print("Volatility DataFrame:\n", volatility_df_list[0].tail(), "\n")
#print("VaR DataFrame:\n", vaR_df_list[0].tail(), "\n")
#print("Momentum DataFrame:\n", momentum_df_list[0].tail(), "\n")
#print("Average Return DataFrame:\n", avg_return_df_list[0].tail(), "\n")

# Print the last 5 rows of the combined DataFrame
features_df.tail()

skew_20                                                         \
           Equities_0 Equities_1 Equities_2 Equities_3 Equities_4 Equities_5   
Column1                                                                        
2023-02-20  -0.240833  -0.198584   0.748803  -0.034554   0.183471  -0.216372   
2023-02-21  -0.086253  -0.070400   0.773535   0.011176   0.312154  -0.041571   
2023-02-22  -0.128589  -0.062873   0.757504  -0.109515   0.415347   0.052482   
2023-02-23  -0.023004   0.106261   0.830820  -0.006954   0.265489   0.092309   
2023-02-24   0.050596   0.138392   0.864532   0.148482   0.428164   0.081861   

                                                        ...  avg_return_480  \
           Equities_6 Equities_7 Equities_8 Equities_9  ... Equity_Sector_1   
Column1                                                 ...                   
2023-02-20   0.256436  -0.157259   1.524128   0.163826  ...        0.000223   
2023-02-21   0.221016  -0.162284   1.521845  -0.296421  ...        0.000203   
2023-02-22   0.191846  -0.470390   1.486008  -0.301710  ...        0.000189   
2023-02-23   0.325509  -0.476984   1.454702  -0.274151  ...        0.000200   
2023-02-24   0.420396  -0.548086   1.506494  -0.256232  ...        0.000202   

                                                                            \
           Equity_Sector_2 Equity_Sector_3 Equity_Sector_4 Equity_Sector_5   
Column1                                                                      
2023-02-20        0.001289        0.000093        0.000232        0.000087   
2023-02-21        0.001335        0.000089        0.000194        0.000060   
2023-02-22        0.001290        0.000056        0.000162        0.000026   
2023-02-23        0.001304        0.000074        0.000161        0.000030   
2023-02-24        0.001269        0.000031        0.000141        0.000001   

                                                                            \
           Equity_Sector_6 Equity_Sector_7 Equity_Sector_8 Equity_Sector_9   
Column1                                                                      
2023-02-20        0.000005        0.000010       -0.000008       -0.000610   
2023-02-21       -0.000026       -0.000004       -0.000074       -0.000646   
2023-02-22       -0.000051       -0.000029       -0.000105       -0.000646   
2023-02-23       -0.000030        0.000023       -0.000123       -0.000627   
2023-02-24       -0.000062        0.000002       -0.000159       -0.000647   

                             
           Equity_Sector_10  
Column1                      
2023-02-20         0.000152  
2023-02-21         0.000083  
2023-02-22         0.000094  
2023-02-23         0.000099  
2023-02-24         0.000093  

[5 rows x 3080 columns]

In [59]:
def RSI(df_returns, window):
    """
    Computes the Relative Strength Index (RSI) for a given window.
    """
    df = df_returns.copy()
    df[df >= 0] = 1
    df[df < 0] = 0
    df = df.rolling(window).mean()*100
    return df

RSI skip for now

In [ ]:
# Initialize an empty dictionary to store the last RSI value for each window
rsi_values = {}

# Calculate RSI for each window and store the last value
for window in [20, 40, 60, 100, 180, 240, 360, 480]:
    rsi_df = RSI(df_last_10_years, window)  # df_returns is your DataFrame with returns data
    last_rsi_value = rsi_df.iloc[-1]  # Get the last row of the RSI DataFrame
    rsi_values[window] = last_rsi_value  # Store it in the dictionary with the window as the key

# Print the last RSI value for a 20-day window
print("Last RSI value for 20-day window:")
print(rsi_values[20])




Forming the DF

In [153]:
# Define the assets and windows outside of the function for clarity
assets = [
    'Equities_0', 'Equities_1', 'Equities_2', 'Equities_3', 'Equities_4', 'Equities_5', 'Equities_6', 'Equities_7',
    'Equities_8', 'Equities_9', 'Equities_10', 'Equities_11', 'Equities_12', 'Equities_13', 'Equities_14', 'Equities_15',
    'Equities_16', 'FX_0', 'FX_1', 'FX_2', 'FX_3', 'FX_4', 'FX_5', 'FX_6', 'FX_7', 'FX_8', 'FX_9', 'FX_10', 'FX_11',
    'FX_12', 'FX_13', 'Bonds_0', 'Bonds_1', 'Bonds_2', 'Bonds_3', 'Bonds_4', 'Bonds_5', 'Bonds_6', 'Bonds_7', 'Bonds_8',
    'Bonds_9', 'Bonds_10', 'Bonds_11', 'Bonds_12', 'Bonds_13', 'Equity_Sector_0', 'Equity_Sector_1', 'Equity_Sector_2',
    'Equity_Sector_3', 'Equity_Sector_4', 'Equity_Sector_5', 'Equity_Sector_6', 'Equity_Sector_7', 'Equity_Sector_8',
    'Equity_Sector_9', 'Equity_Sector_10'
]
windows = [20, 40, 60, 100, 180, 240, 360, 480]

# Generate the final DataFrame
final_rows = []
for date in df_last_10_years.index:
    for asset in assets:
        row = [date, asset]
        for feature_name, feature_dict in [('skew', skew), ('kurtosis', kurtosis), ('max_drawdown', max_drawdown), 
                                           ('volatility', volatility), ('vaR', vaR), ('momentum', momentum), 
                                           ('avg_return', avg_return)]:
            for window in windows:
                # Check if the window exists for this feature, if not, use NaN
                value = feature_dict[window].loc[date, asset] if window in feature_dict else float('nan')
                row.append(value)
        final_rows.append(row)

# Define the column names for the final DataFrame
column_names = ['Date', 'Asset']
for feature_name in ['skew', 'kurtosis', 'max_drawdown', 'volatility', 'vaR', 'momentum', 'avg_return']:
    for window in windows:
        column_names.extend([f'{feature_name}_{window}'])

# Now create the DataFrame
final_df = pd.DataFrame(final_rows, columns=column_names)

# Show the first few rows of the DataFrame
print(final_df.tail())


            Date             Asset   skew_20   skew_40   skew_60  skew_100  \
46027 2023-02-24   Equity_Sector_6  0.526400  0.257828  0.081595  0.827986   
46028 2023-02-24   Equity_Sector_7  0.256501  0.495935  0.103227  0.486835   
46029 2023-02-24   Equity_Sector_8  0.818443  0.520856  0.358629  1.111640   
46030 2023-02-24   Equity_Sector_9  1.429924  0.867425  0.432064  0.348426   
46031 2023-02-24  Equity_Sector_10 -0.185238 -0.284520 -0.171236  0.250022   

       skew_180  skew_240  skew_360  skew_480  ...  momentum_360  \
46027  0.395703  0.093499  0.083147  0.024990  ...     -0.115211   
46028  0.442083  0.166631  0.107450  0.082749  ...      0.023760   
46029  0.656045  0.191849  0.122967  0.063018  ...     -0.150483   
46030  0.190163  0.044191 -0.114346 -0.165398  ...     -0.373065   
46031 -0.052785 -0.211851 -0.250760 -0.252899  ...      0.061617   

       momentum_480  avg_return_20  avg_return_40  avg_return_60  \
46027     -0.029955      -0.001087       0.001747     

NN - model free

In [175]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Define the neural network model
class MultivariateNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MultivariateNN, self).__init__()
        self.input_layer = nn.Linear(input_dim, hidden_dim)
        self.hidden_layer = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x = F.leaky_relu(self.input_layer(x))
        x = F.softmax(self.hidden_layer(x))
        return x

# Custom Sharpe Ratio Loss
class SharpeRatioLoss(nn.Module):
    def __init__(self, risk_free_rate=0):
        super(SharpeRatioLoss, self).__init__()
        self.risk_free_rate = risk_free_rate

    def forward(self, outputs):
        expected_return = outputs.mean()
        std_dev_return = outputs.std()
        sharpe_ratio = (expected_return - self.risk_free_rate) / (std_dev_return + 1e-6)
        return -sharpe_ratio

# Assuming 'feature_df' is your dataset as a pandas DataFrame
# Calculated features must be part of 'feature_df'
calculated_features_df = pd.DataFrame(feature_df)
features_df = calculated_features_df.fillna(calculated_features_df.mean())

#test
returns = df_last_10_years.drop(columns=df_last_10_years.columns[0])

# Split the data into training and testing sets (considering all columns for y)
X_train, X_test, y_train, y_test = train_test_split(features_df, returns, test_size=0.2, random_state=42)

"""# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_df, features_df, test_size=0.2, random_state=42)
"""

# Scaling the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Converting to PyTorch tensor
X_train_tensor = torch.FloatTensor(X_train_scaled)
X_test_tensor = torch.FloatTensor(X_test_scaled)

# Initialize the neural network model
#model = MultivariateNN(X_train.shape[1], y_train.shape[1])
input_dim = X_train_tensor.shape[1] # Specify the number of input features
hidden_dim = 32  # Specify the number of neurons in the hidden layer
#output_dim = 55  # Specify the number of assets or allocation decisions
output_dim = y_train.shape[1]  # Number of columns to predict
print(input_dim)
model = MultivariateNN(input_dim, hidden_dim, output_dim)

# Loss and optimizer
criterion = SharpeRatioLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)


# Training loop
epochs = 50
for epoch in tqdm(range(epochs)):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs)
    loss.backward()
    optimizer.step()

    # Optional: Print loss every N epochs
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

# Extracting only the weights (and biases, if needed) of the output layer
output_layer_weights = model.hidden_layer.weight.data.cpu().numpy()
output_layer_biases = model.hidden_layer.bias.data.cpu().numpy()

# You can now use output_layer_weights and output_layer_biases as needed
print("Output Layer Weights:", output_layer_weights)
print("Output Layer Biases:", output_layer_biases)

# Evaluate the model
model.eval()
with torch.no_grad():  # Turn off gradients for validation, saves memory and computations
    neural_network_output = model(X_test_tensor).cpu().numpy()
# Now neural_network_output contains the output of the neural network
print("Neural Network Output:", neural_network_output)
##print the dimensions of neural network
print("Neural Network Output shape:", neural_network_output.shape)
#with torch.no_grad():
    #train_outputs = model(X_train_tensor)
    #test_outputs = model(X_test_tensor)
    #train_loss = criterion(train_outputs)
    #test_loss = criterion(test_outputs)
    #print(f"Final Training Loss: {train_loss.item()}")
    #print(f"Final Test Loss: {test_loss.item()}")





448


/var/folders/wn/5q1p8b8n0992yr7qgp1s8pv00000gn/T/ipykernel_4863/1725631932.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(self.hidden_layer(x))


Epoch 10/50, Loss: -2.028352975845337
Epoch 20/50, Loss: -4.184818267822266
Epoch 30/50, Loss: -6.348658561706543


100%|██████████| 50/50 [00:00<00:00, 85.33it/s]

Epoch 40/50, Loss: -8.84572696685791
Epoch 50/50, Loss: -14.55651569366455
Output Layer Weights: [[ 0.06030712  0.06034895  0.06021939 ...  0.04485895 -0.04009866
   0.10348577]
 [ 0.07706785  0.02154174 -0.12302779 ... -0.00113791  0.12559265
   0.1308346 ]
 [ 0.06152293  0.12568976 -0.00150169 ...  0.01377312  0.02661596
   0.13587077]
 ...
 [ 0.04617379  0.07505186  0.18688197 ...  0.0413616  -0.05733545
   0.0743194 ]
 [ 0.02462356  0.08353397  0.03387302 ...  0.05489655  0.03614821
   0.03483846]
 [ 0.14545876  0.07126361  0.11199585 ... -0.00419998 -0.07389294
   0.20505956]]
Output Layer Biases: [ 0.01276876  0.02018515 -0.04689267 -0.01022161  0.00157323 -0.01175919
  0.02269167  0.0168194   0.00170925  0.0288313  -0.03933667  0.03971092
 -0.01995482  0.03591099 -0.01149204 -0.02274209 -0.00681519  0.00722248
  0.02638999 -0.03805919  0.05720951  0.01127013  0.00728741 -0.04639073
 -0.04476226  0.00416599 -0.0114892  -0.00848199 -0.01001663  0.02045355
 -0.0287329   0.00249665 

NN - Model Based

In [177]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import pandas as pd
import cvxpy as cp
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from joblib import Parallel, delayed

# Define the neural network model
class MultivariateNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MultivariateNN, self).__init__()
        self.input_layer = nn.Linear(input_dim, hidden_dim)
        self.hidden_layer = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x = F.leaky_relu(self.input_layer(x))
        x = F.softmax(self.hidden_layer(x))
        return x

# Custom Sharpe Ratio Loss
class SharpeRatioLoss(nn.Module):
    def __init__(self, risk_free_rate=0):
        super(SharpeRatioLoss, self).__init__()
        self.risk_free_rate = risk_free_rate

    def forward(self, outputs):
        expected_return = outputs.mean()
        std_dev_return = outputs.std()
        sharpe_ratio = (expected_return - self.risk_free_rate) / (std_dev_return + 1e-6)
        return -sharpe_ratio

calculated_features_df = pd.DataFrame(feature_df)
features_df = calculated_features_df.fillna(calculated_features_df.mean())
returns = df_last_10_years.drop(columns=df_last_10_years.columns[0])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_df, returns, test_size=0.2, random_state=42)

# Scaling the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Converting to PyTorch tensor
X_train_tensor = torch.FloatTensor(X_train_scaled)
X_test_tensor = torch.FloatTensor(X_test_scaled)

# Initialize the neural network model
input_dim = X_train_tensor.shape[1]
hidden_dim = 32
output_dim = y_train.shape[1]
model = MultivariateNN(input_dim, hidden_dim, output_dim)

# Loss and optimizer
criterion = SharpeRatioLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Define the optimization function
def optimize_weights(t, returns, n_assets, initial_allocations):
    b = np.ones(n_assets) / n_assets  # Equal risk budgeting
    c = 1  # Constant for constraint
    data_t = returns.iloc[:t]
    cov_matrix_values = data_t.cov().values
    cov_matrix_values = (cov_matrix_values + cov_matrix_values.T) / 2
    y = cp.Variable(shape=n_assets)
    objective = cp.Minimize(cp.sqrt(cp.quad_form(y, cp.psd_wrap(cov_matrix_values))))
    constraints = [
        cp.sum(cp.multiply(b, cp.log(y))) >= c,
        y >= 1e-5
    ]
    problem = cp.Problem(objective, constraints)
    problem.solve(solver=cp.SCS, qcp=True, eps=1e-5, max_iters=100)
    optimal_weights = y.value
    return optimal_weights

# Training loop with integrated optimization
epochs = 50
for epoch in tqdm(range(epochs)):
    model.train()
    optimizer.zero_grad()

    # Forward pass through the neural network
    nn_outputs = model(X_train_tensor)

    # Optimize the weights for each time step
    optimized_weights = []
    for t in range(len(X_train)):
        initial_allocations = nn_outputs[t].detach().numpy()
        optimized_weights.append(optimize_weights(t, returns, output_dim, initial_allocations))

    # Calculate the loss based on the optimized weights
    loss = criterion(torch.tensor(optimized_weights))

    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")

# Evaluate the model
model.eval()
with torch.no_grad():
    neural_network_output = model(X_test_tensor).cpu().numpy()
    print("Neural Network Output:", neural_network_output)
    print("Neural Network Output shape:", neural_network_output.shape)




/var/folders/wn/5q1p8b8n0992yr7qgp1s8pv00000gn/T/ipykernel_4863/4215322618.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.softmax(self.hidden_layer(x))
/Users/jakobamaya-scott/opt/anaconda3/envs/AP1/lib/python3.11/site-packages/numpy/lib/function_base.py:520: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/Users/jakobamaya-scott/opt/anaconda3/envs/AP1/lib/python3.11/site-packages/numpy/core/_methods.py:121: RuntimeWarning: divide by zero encountered in divide
  ret = um.true_divide(
/Users/jakobamaya-scott/opt/anaconda3/envs/AP1/lib/python3.11/site-packages/numpy/core/_methods.py:121: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/Users/jakobamaya-scott/opt/anaconda3/envs/AP1/lib/python3.11/site-packages/pandas/core/frame.py:10869: RuntimeWarning: Degrees of freedom <= 0 for slice
  base_cov = np.cov(mat.T, ddof=ddof)
/Users/jakobamaya-scott/o

ValueError: array must not contain infs or NaNs

AP1 risk budgetering

In [164]:
import cvxpy as cp
from tqdm import tqdm
from joblib import Parallel, delayed
# Assuming df_last_10_years contains the daily returns
returns = df_last_10_years.drop(columns=df_last_10_years.columns[0])
n_assets = len(returns.columns)
# Define a function that will be parallelized
def optimize_weights(t, returns, n_assets):
    b = np.ones(n_assets) / n_assets  # For example, equal risk budgeting
    c = 1
    # Code that was originally in your for-loop goes here
    # For example:
    data_t = returns.iloc[:t]
    cov_matrix_values = data_t.cov().values
    cov_matrix_values = (cov_matrix_values + cov_matrix_values.T)/2
    y = cp.Variable(shape=n_assets)
    # Objective function: Minimize the square root of the portfolio variance
    objective = cp.Minimize(cp.sqrt(cp.quad_form(y, cp.psd_wrap(cov_matrix_values))))
    constraints = [
        cp.sum(cp.multiply(b, cp.log(y))) >= c,
        y >= 1e-5 #strict inequalities are not allowed
    ]
    # Formulate the optimization problem
    problem = cp.Problem(objective, constraints)
    # Solve the problem using a suitable solver
    problem.solve(solver=cp.SCS,qcp=True, eps = 1e-5, max_iters  = 100) 

    # Extract the results
    optimal_weights = y.value
    date = data_t.index[-1]
    # Return the results for this iteration
    return (date, optimal_weights)

# Precompute any variables that don't change inside the loop
# ...

# Set up the joblib parallelization
# Here, 'range(len(returns))' is the range over which you want to parallelize
# results = Parallel(n_jobs=-1)(delayed(optimize_weights)(t, returns, n_assets) for t in tqdm(range(54, len(returns))))
# Set up the joblib parallelization with tqdm
results = Parallel(n_jobs=-1)(delayed(optimize_weights)(t, returns, n_assets) for t in tqdm(range(54, len(returns), 5))
)
# After parallelization, recombine the results as necessary
# For example:
# Create a dictionary with dates as keys and optimal weights as values
optimal_weights_dict = {date: weights for date, weights in results}

w = pd.DataFrame.from_dict(optimal_weights_dict, orient='index', columns=returns.columns)
# Normalize the weights
w = w.div(w.sum(axis=1), axis=0)
# Calculate the portfolio returns
portfolio_returns = w.shift(1).mul(returns).dropna(how='all')
portfolio_returns.sum(axis=1).cumsum().plot()

 13%|█▎        | 20/154 [01:13<08:27,  3.79s/it]

KeyboardInterrupt: 